In [1]:
import re
import time
from config import username, password
from splinter import Browser
from Models.DB import DB

In [2]:
db = DB()
db.connect()
db.createTableReviews("indeed")

You are connected to -  ['PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit'] 

(['companyreviews', 'public', 'glassdoor_reviews', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None], ['companyreviews', 'public', 'indeed_reviews', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None])


In [1]:
#db.dropTables("indeed_reviews")

In [3]:
urls = ["https://secure.indeed.com/account/login?hl=en_US&co=US&continue=https%3A%2F%2Fwww.indeed.com%2Fcmp%2FDaugherty-Business-Solutions%2Freviews&tmpl=desktop&service=&from=gnav-util-acme--acme-webapp",
        "https://www.indeed.com/cmp/Daugherty-Business-Solutions/reviews"]

In [4]:
def login(url=urls[0]):
    pass

In [5]:
def visitURL(url=urls[1]):
    global browser
    
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.driver.set_window_size(1280, 720)
    
    browser.visit(url)

In [6]:
visitURL(url=urls[1])

In [21]:
def scrapePage(_id, browser=browser):

    reviews = browser.find_by_css('div[class="cmp-Review-container"]')

    for review in reviews:

        _id += 1

        header = review.find_by_css('div[class="cmp-Review-title"]').text.replace("%", "%%").replace("$", "\$")
        body = review.find_by_css('span[itemprop="reviewBody"]').text.replace("%", "%%").replace("$", "\$")
        rating = float(review.find_by_css('div[class="cmp-ReviewRating-text"]').text)
        role = review.find_by_css('a[class="cmp-ReviewAuthor-link"]').text.replace("%", "%%").replace("$", "\$")

        try:
            date = review.find_by_css('span[class="cmp-ReviewAuthor"]').text
            date = re.sub(".*?(?=(\w+ \d+, \d+))", "", date) # returns only date

        except:
            date = "0"

        source = "indeed"

        response = "No response"


        print(f"Id: {_id}\nDate: {date}\n{header}\n{rating}\n{body}\n{source}\nResponse: {response}\n-----------------------------------------------------------------------------")

        db.intoReviews("indeed", header, body, rating, role, date, response, source)

In [22]:
def scrapeReviews():
    """
        header, body, rating, role, date, response, source
    """ 
    _id = 0
    
    while browser.is_element_present_by_css('a[data-tn-element="next-page"]'):
        
        button = browser.find_by_css('a[data-tn-element="next-page"]')
        
        scrapePage(_id)
            
        #simulates clicking "next" button
        browser.visit( button._element.get_attribute("href"))
        
    scrapePage(_id)

In [ ]:
scrapeReviews()